In [1]:
cd ..

/home/vamato/distributed-energy-systems


In [2]:
from osmnx import ts
import yaml
from pathlib import Path
import os
import pickle
import pypsa
import pandas as pd

import re
from plotly.subplots import make_subplots
from tqdm import tqdm
import plotly.graph_objs as go
import plots.plots as pt


def read_parameters():
    params_path = "plots/parameters.yaml"

    with open(params_path, "r") as f:
        parameters = yaml.safe_load(f)

    return parameters['countries'], parameters['yearly'], parameters['monthly'], parameters['weekly'], parameters['centralized']


def get_metadata_from_folder(folder_path):
    # Get years and timeslices from the folder structure
    folder_path = os.path.join(folder_path, 'DCOP')
    years = [int(year) for year in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, year))]
    years.sort()
    # Assume all years have the same timeslices, get from the first year
    first_year_folder = os.path.join(folder_path, str(years[0]))
    timeslices = [ts for ts in os.listdir(first_year_folder) if os.path.isdir(os.path.join(first_year_folder, ts))]
    timeslices.sort()

    return years, timeslices

def get_data_from_folder(folder_path, extra, countries):

    years, timeslices = get_metadata_from_folder(folder_path)

    dataframes = {}
    folder_path = os.path.join(folder_path, 'DCOP')
    for year in years:
        dataframes[year] = {}
        year_folder = os.path.join(folder_path, str(year))
        for timeslice in timeslices:
            dataframes[year][timeslice] = {}
            timeslice_folder = os.path.join(year_folder, timeslice)
            if not os.path.exists(timeslice_folder):
                raise FileNotFoundError(f"No data for year {year} and timeslice {timeslice} in folder {timeslice_folder}")
            else:
                for file_k in os.listdir(timeslice_folder):
                    if file_k.endswith('.pkl'):
                        dataframes[year][timeslice][file_k.split('_')[1]] = {}
                        file_path = os.path.join(timeslice_folder, file_k)
                        with open(file_path, 'rb') as f:
                            data = pickle.load(f)
                            if isinstance(data, dict):
                                for key, value in data.items():
                                    dataframes[year][timeslice][file_k.split('_')[1]][key] = value
                            else:
                                raise ValueError(f"Data in {file_k} is not a dict")
                            f.close()

    networks = {}
    for year in years:
        networks[year] = {}
        for timeslice in tqdm(timeslices, desc=f"Processing timeslices for year {year}"):
            ts_data = dataframes[year][timeslice]
            k_keys_sorted = sorted(ts_data.keys(), key=lambda x: int(x))
            for k in k_keys_sorted:
                k_data = ts_data[k]
                k_data['df']['exchange_cost'] = k_data['df']['total_cost_after_exchange'] - k_data['df']['0']
                dataframes[year][timeslice][k] = k_data

            cost_with_exchange = {country: 0 for country in dataframes[year][timeslice][k]['df'].index}
            k_keys_sorted = sorted(ts_data.keys(), key=lambda x: int(x))
            for k in k_keys_sorted:
                df = dataframes[year][timeslice][k]['df']
                for country in df.index:
                    cost = df.loc[country, '0'] if '0' in df.columns else None
                    cost_with_exchange[country] += df.loc[country, 'exchange_cost'] if 'exchange_cost' in df.columns else 0
                    df.loc[country, 'total_cost_after_exchange'] = cost + cost_with_exchange[country] * 0.95

            networks[timeslice] = {}
            rows = []
            for c in countries:
                network_path = os.path.join(
                    'pypsa_earth_before_weekly_rerun', 
                    'results', 
                    f'{year}_{timeslice}_{c}___{folder_path.replace(extra, "").replace("solutions/", "")}', 
                    'networks'
                ).replace('DCOP/', '')
                network_files = [f for f in os.listdir(network_path) if f.endswith('.nc')]
                if network_files:
                    if len(network_files) > 1:
                        with open(os.path.join(network_path, "latest.txt"), "r") as latest_file:
                            latest_network_file = latest_file.readline().strip()
                        network_path = os.path.join(network_path, latest_network_file)
                    else:
                        network_path = os.path.join(network_path, network_files[0])
                if os.path.exists(network_path):
                    networks[timeslice][c] = pypsa.Network(network_path)
                    n = networks[timeslice][c]
                    for (component, technology), row in n.statistics().iterrows():
                        if component not in ["Generator", "Link", "StorageUnit", "Store"]:
                            continue
                        rows.append({
                            "technology": technology,
                            "demand_type": "0",
                            "country": c,
                            "Optimal Capacity": row.get("Optimal Capacity", 0),
                            "Installed Capacity": row.get("Installed Capacity", 0),
                            "Supply": row.get("Supply", 0),
                            "Capital Expenditure": row.get("Capital Expenditure", 0),
                            "Operational Expenditure": row.get("Operational Expenditure", 0),
                            "Capacity Factor": row.get("Capacity Factor", 0),
                            "Curtailment": row.get("Curtailment", 0),
                        })

                else:
                    print(f"Network file not found for timeslice {timeslice}: {network_path}")
            tech_df = pd.DataFrame(rows, columns=['technology', 'demand_type', 'country', 'Optimal Capacity', 'Installed Capacity', 'Supply', 'Capital Expenditure', 'Operational Expenditure', 'Capacity Factor', 'Curtailment'])
            # Get the last k key from dataframes[year][timeslice]
            k = sorted(dataframes[year][timeslice].keys(), key=lambda x: int(x))[-1]
            dataframes[year][timeslice][k]['tech_df'] = tech_df
    return dataframes, networks

In [3]:
countries, yearly, monthly, weekly, centralized = read_parameters()

dataframes_yearly, networks_yearly = get_data_from_folder(yearly['folder'], yearly['extra'], countries)
dataframes_monthly, networks_monthly = get_data_from_folder(monthly['folder'], monthly['extra'], countries)
dataframes_weekly, networks_weekly = get_data_from_folder(weekly['folder'], weekly['extra'], countries)
#dataframes_centralized = get_data_from_folder(centralized['folder'])

Processing timeslices for year 2030:   0%|          | 0/1 [00:00<?, ?it/s]INFO:pypsa.io:Imported network elec_s_10flex_ec_lcopt_Co2L0.36950322716171624.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_10flex_ec_lcopt_Co2L0.0040140228029353865.nc has buses, carriers, generators, global_constraints, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_10flex_ec_lcopt_Co2L-2H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_10flex_ec_lcopt_Co2L-2H.nc has buses, carriers, generators, global_constraints, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_10flex_ec_lcopt_Co2L-2H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_10flex_ec_lcopt_Co2L-2H.nc has buses, carriers, generators, global_cons

In [4]:
import pypsa
solved_network_path = f"./pypsa_earth_before_weekly_rerun/results/SAPP-centralized-2h-120flex-363mt/networks/elec_s_120flex_ec_lcopt_Co2L-2H.nc"

if not os.path.exists(solved_network_path):
    raise FileNotFoundError(f"Network file {solved_network_path} does not exist.")
    
n = pypsa.Network(solved_network_path)

statistics = n.statistics()
total_cost = statistics["Capital Expenditure"].sum() + statistics["Operational Expenditure"].sum()

rows = []
for (component, technology), row in n.statistics().iterrows():
    if component not in ["Generator", "Link", "StorageUnit", "Store"]:
        continue
    rows.append({
        "technology": technology,
        "country": "SAPP",
        "Optimal Capacity": row.get("Optimal Capacity", 0),
        "Installed Capacity": row.get("Installed Capacity", 0),
        "Supply": row.get("Supply", 0),
        "Capital Expenditure": row.get("Capital Expenditure", 0),
        "Operational Expenditure": row.get("Operational Expenditure", 0),
        "Capacity Factor": row.get("Capacity Factor", 0),
        "Curtailment": row.get("Curtailment", 0),
    })
tech_df = pd.DataFrame(rows, columns=['technology','country', 'Optimal Capacity', 'Installed Capacity', 'Supply', 'Capital Expenditure', 'Operational Expenditure', 'Capacity Factor', 'Curtailment'])

dataframes_centralized = {2030: {'C': {'0': {'k': 0, 't': 'C', 'year': 2030, 'tech_df': tech_df}}}}

INFO:pypsa.io:Imported network elec_s_120flex_ec_lcopt_Co2L-2H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [5]:
unsolved_network_path = f"./pypsa_earth/networks/SAPP-centralized-2h-120flex-363mt/elec_s_120flex_ec_lcopt_Co2L-2H.nc"

unsolved_n = pypsa.Network(unsolved_network_path)
# Only keep components of interest for initial_capacity_df
initial_capacity_df = unsolved_n.statistics()['Installed Capacity']
initial_capacity_df = initial_capacity_df.loc[
    [comp in ["Generator", "Link", "StorageUnit", "Store"] for comp, _ in initial_capacity_df.index]
]
initial_capacity_df.index = initial_capacity_df.index.get_level_values(1)
initial_capacity_df = initial_capacity_df[initial_capacity_df.index != "DC"]
initial_capacity_df

INFO:pypsa.io:Imported network elec_s_120flex_ec_lcopt_Co2L-2H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


Biomass                    79.83700
Coal                    49373.73159
Combined-Cycle Gas       6593.15773
Nuclear                  1940.00000
Oil                       532.43902
Onshore Wind             3416.46669
Run of River             6075.69869
Solar                    6607.03807
Pumped Hydro Storage     2892.00000
Reservoir & Dam         13995.92357
Name: Installed Capacity, dtype: float64

In [6]:
SAVING_PATH = "./plots/images"

def try_int(x):
    try:
        return int(x)
    except Exception:
        return x

def timeslice_sort_key(ts):
    match = re.match(r"[MW](\d+)", str(ts))
    return int(match.group(1)) if match else ts

def get_years_and_timeslices_from_dataframes(dataframes):
    years = sorted(dataframes.keys())
    if not years:
        return [], []
    # Get timeslices from the first year
    first_year = years[0]
    timeslices = sorted(dataframes[first_year].keys())
    return years, timeslices

def extract_k_max_per_timeslice(timeslices, dataframes, YEAR):
    k_max_per_timeslice = {}
    for ts in timeslices:
        if ts in dataframes[YEAR]:
            k_values = [ts_data['k'] for ts_data in dataframes[YEAR][ts].values() if 'k' in ts_data]
        if k_values:
            k_max_per_timeslice[ts] = max(k_values)
        else:
            k_max_per_timeslice[ts] = None  # or np.nan if you prefer

    # Sort timeslices and corresponding k_max values using timeslice_sort_key
    sorted_items = sorted(k_max_per_timeslice.items(), key=lambda item: timeslice_sort_key(item[0]))
    #x_vals = [item[0] for item in sorted_items]
    #y_vals = [item[1] for item in sorted_items]

    return sorted_items



In [7]:
# 🎨 Cleaned Tech Color Palette
palette = {
    "onshore_wind": "#235ebc",
    "offshore_wind_ac": "#6895dd",
    "offshore_wind_dc": "#74c6f2",
    "wave": "#004444",
    "hydro": "#08ad97",
    "run_of_river": "#4adbc8",
    "solar_pv": "#f9d002",
    "solar_thermal": "#ffef60",
    "biomass": "#0c6013",
    "solid_biomass": "#06540d",
    "solid_biomass_cc": "#654321",
    "biogas": "#23932d",
    "waste": "#68896b",
    "geothermal": "#ba91b1",
    "ocgt": "#d35050",
    "ocgt_heat": "#ee8340",
    "ocgt_marginal": "sandybrown",
    "ccgt": "#b80404",
    "gas_boiler": "#ee8340",
    "gas_to_power_heat": "brown",
    "smr": "#4F4F2F",
    "smr_cc": "darkblue",
    "oil": "#262626",
    "oil_boiler": "#B5A642",
    "oil_emissions": "#666666",
    "gas_industry": "#333333",
    "nuclear": "#ff9000",
    "coal": "#707070",
    "lignite": "#9e5a01",
    "h2": "#ea048a",
    "h2_industry": "#222222",
    "h2_shipping": "#6495ED",
    "h2_liquefaction": "m",
    "battery": "slategray",
    "home_battery": "#614700",
    "ac_lines": "#70af1d",
    "dc_links": "#8a1caf",
    "load": "#ff0000",
    "electricity": "k",
    "grid": "y",
    "heat": "darkred",
    "heat_pump_air": "#76EE00",
    "heat_pump_ground": "#40AA00",
    "rural_heat": "#880000",
    "central_heat": "#b22222",
    "decentral_heat": "#800000",
    "low_temp_heat_industry": "#991111",
    "process_heat": "#FF3333",
    "power_to_heat": "red",
    "resistive_heater": "pink",
    "methanation": "#FF1493",
    "power_to_gas": "purple",
    "power_to_liquid": "darkgreen",
    "helmeth": "#7D0552",
    "dac": "deeppink",
    "co2_stored": "#123456",
    "co2_vent": "#654321",
    "process_emissions": "#222222",
    "process_emissions_cc": "gray",
    "process_emissions_stored": "#444444",
    "process_emissions_atm": "#888888",
    "agriculture_heat": "#D07A7A",
    "agriculture_machinery_oil": "#1e1e1e",
    "agriculture_machinery_oil_emissions": "#111111",
    "agriculture_electricity": "#222222",
    "fischer_tropsch": "#44DD33",
    "kerosene_aviation": "#44BB11",
    "naphtha_industry": "#44FF55",
    "land_transport_oil": "#44DD33",
    "land_transport_oil_emissions": "#666666",
    "land_transport_fc": "#AAAAAA",
    "land_transport_ev": "grey",
    "v2g": "grey",
    "bev_charger": "grey",
    "shipping": "#6495ED",
    "water_tanks": "#BBBBBB",
    "hot_water_storage": "#BBBBBB",
    "hot_water_charging": "#BBBBBB",
    "hot_water_discharging": "#999999",
    "li_ion": "grey",
    "district_heating": "#CC4E5C",
    "retrofitting": "purple",
    "solid_biomass_transport": "green",
    "biomass_eop": "green",
    "high_temp_electrolysis": "magenta",
    "today": "#D2691E",
    "ambient": "k",
}

# ✅ Nice Names for Key Techs
nice_names = {
    "OCGT": "Open-Cycle Gas",
    "CCGT": "Combined-Cycle Gas",
    "offshore_wind_ac": "Offshore Wind (AC)",
    "offshore_wind_dc": "Offshore Wind (DC)",
    "onshore_wind": "Onshore Wind",
    "solar_pv": "Solar",
    "PHS": "Pumped Hydro Storage",
    "hydro": "Reservoir & Dam",
    "battery": "Battery Storage",
    "h2": "Hydrogen Storage",
    "ac_lines": "Transmission Lines",
    "run_of_river": "Run of River",
    "battery charger": "Battery Charger",
    "battery discharger": "Battery Discharger",
}

custom_palette = {
    'Battery Storage': palette.get('battery', 'grey'),
    'Biomass': palette.get('biomass', 'green'),
    'Coal': palette.get('coal', 'black'),
    'Combined-Cycle Gas': palette.get('CCGT', palette.get('ccgt', 'red')),
    'DC': palette.get('dc_links', 'purple'),
    'H2 electrolysis': palette.get('high_temp_electrolysis', 'magenta'),
    'H2 fuel cell': palette.get('h2_fuel_cell', 'cyan'),
    'Hydrogen Storage': palette.get('Hydrogen Storage', 'pink'),
    'Nuclear': palette.get('nuclear', 'orange'),
    'Oil': palette.get('oil', 'brown'),
    'Onshore Wind': palette.get('Onshore Wind', 'blue'),
    'Pumped Hydro Storage': palette.get('PHS', palette.get('hydro', 'aqua')),
    'Reservoir & Dam': palette.get('Reservoir & Dam', 'teal'),
    'Run of River': palette.get('Run of River', 'lightblue'),
    'Solar': palette.get('Solar', palette.get('solar_pv', 'yellow')),
    'battery charger': palette.get('battery charger', 'slategray'),
    'battery discharger': palette.get('battery discharger', 'slategray'),
}

In [8]:
import pandas as pd
import plotly.express as px
import math
import numpy as np

# ---------- helpers you can reuse ----------
def _pick_last_k(block: dict):
    def _key(x):
        try:
            return int(x)
        except Exception:
            return x
    return sorted(block.keys(), key=_key)[-1]

def _clean_tech_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if 'demand_type' in df.columns:
        df['demand_type'] = 0
        df = df.drop(columns=['demand_type'])
    col_map = {}
    for c in df.columns:
        lc = c.strip().lower()
        if lc in ('technology', 'tech', 'technology_name'):
            col_map[c] = 'technology'
        elif lc in ('installed capacity', 'installed_capacity', 'capacity_installed'):
            col_map[c] = 'Installed Capacity'
        elif lc in ('country', 'region', 'node'):
            col_map[c] = 'country'
    df = df.rename(columns=col_map)
    if not {'technology', 'Installed Capacity'}.issubset(df.columns):
        raise ValueError(f"tech_df missing required cols. Have: {list(df.columns)}")
    df['technology'] = df['technology'].astype(str)
    df = df[~df['technology'].str.lower().eq('load')]
    df['Installed Capacity'] = pd.to_numeric(df['Installed Capacity'], errors='coerce').fillna(0.0)
    return df

def _capacity_per_tech_for_year(full_dataset: dict, year) -> pd.DataFrame:
    """
    full_dataset[year][timeslice][k]['tech_df'] -> DataFrame
    For the given year:
      - iterate all timeslices
      - within each timeslice use last/highest k
      - sum Installed Capacity by technology across countries
    Then take MAX across timeslices per technology.
    Returns columns: ['technology', 'Installed Capacity']
    """
    if year not in full_dataset:
        raise KeyError(f"Year '{year}' not found. Available: {list(full_dataset.keys())}")
    per_slice = []
    for ts, k_block in full_dataset[year].items():
        if not isinstance(k_block, dict) or not k_block:
            continue
        k = _pick_last_k(k_block)
        tech_df = k_block[k].get('tech_df')
        if tech_df is None or not isinstance(tech_df, pd.DataFrame) or tech_df.empty:
            continue
        df = _clean_tech_df(tech_df)
        # Do NOT sum by technology; keep per-country division
        df['__timeslice__'] = ts
        per_slice.append(df)
    if not per_slice:
        raise ValueError(f"No valid tech_df data found for year {year}.")
    conc = pd.concat(per_slice, ignore_index=True)
    
    return conc

def _capacities_long_from_full(*, dataframes_yearly=None, dataframes_monthly=None, dataframes_weekly=None, dataframes_centralized=None, year=None):
    """Return a long df with columns: dataset, technology, Installed Capacity."""
    provided = {
        'yearly': dataframes_yearly,
        'monthly': dataframes_monthly,
        'weekly': dataframes_weekly,
        'centralized': dataframes_centralized
    }
    provided = {k: v for k, v in provided.items() if v is not None}
    if not provided:
        raise ValueError("Provide at least one of dataframes_monthly/weekly/centralized.")

    rows = []
    for name, ds in provided.items():
        tb = _capacity_per_tech_for_year(ds, year)
        if not tb.empty:
            tb = tb.copy()
            tb['dataset'] = name
            rows.append(tb)
    if not rows:
        raise ValueError(f"No valid tech_df data found for year {year} in any dataset.")
    result = pd.concat(rows, ignore_index=True)
    # Remove rows where technology is 'DC' (case-insensitive)
    result = result[~result['technology'].str.lower().isin(['dc', 'hydrogen storage'])]
    return result


# ---------- main: better comparison plots ----------
def build_better_comparison_plots(
    *,
    dataframes_yearly=None,
    dataframes_monthly=None,
    dataframes_weekly=None,
    dataframes_centralized=None,
    year=None,
    top_n_for_slope: int = 8,
    baseline: str = 'centralized',
    initial_capacity_df=None,
):
    """
    Returns a dict of Plotly figures:
      - 'composition_share': 100% stacked composition per dataset (tech shares)
      - 'delta_vs_baseline': diverging bars of (dataset - baseline) by technology (sorted by |delta|)
      - 'slope_topN': lines across datasets for the top-N technologies with biggest deltas
    """
    df = _capacities_long_from_full(
        dataframes_yearly=dataframes_yearly,
        dataframes_monthly=dataframes_monthly,
        dataframes_weekly=dataframes_weekly,
        dataframes_centralized=dataframes_centralized,
        year=year
    )
    if df.empty:
        return {'composition_share': None, 'delta_vs_baseline': None, 'slope_topN': None}

    # consistent colors by technology
    # Use lower-case technology names if not found in palette, otherwise use as-is
    tech_colors = {}
    for tech in df['technology'].unique():
        if tech in custom_palette:
            tech_colors[tech] = custom_palette[tech]
        else:
            tech_colors[tech] = custom_palette.get(tech.lower(), '#888888')

    # dataset order (baseline first if present)
    all_datasets = df['dataset'].unique().tolist()
    if baseline not in all_datasets:
        baseline = all_datasets[0]
    dataset_order = [baseline] + [d for d in ['yearly', 'monthly', 'weekly', 'centralized'] if d in all_datasets and d != baseline] + \
                    [d for d in all_datasets if d not in ['yearly', 'monthly', 'weekly', 'centralized', baseline]]

    # ---------- 1) 100% stacked composition ----------
    # For each (dataset, technology, __timeslice__), keep the row with the highest Installed Capacity (by __timeslice__)
    idx = (
        df.groupby(['dataset', 'technology', '__timeslice__'])['Optimal Capacity']
        .idxmax()
        .dropna()
        .astype(int)
    )
    filtered_df = df.loc[idx].reset_index(drop=True)
    cap_df = filtered_df.groupby(['dataset', 'technology'], as_index=False)['Optimal Capacity'].sum()

    # # --- Add initial_capacity_df as a baseline for comparison ---
    # if initial_capacity_df is not None:
    #     # Convert initial_capacity_df to DataFrame
    #     init_df = initial_capacity_df.reset_index()
    #     init_df.columns = ['technology', 'Initial Capacity']
    #     init_df['dataset'] = 'initial'
    #     cap_df = pd.concat([cap_df, init_df[['dataset', 'technology', 'Initial Capacity']]], ignore_index=True)
    #     # Ensure initial is first in order
    #     if 'initial' not in dataset_order:
    #         dataset_order = ['initial'] + dataset_order

    comp_title = f"Installed Capacity by Technology — Year {year}"
    fig_comp_absolute = px.bar(
        cap_df,
        x='dataset', y='Optimal Capacity', color='technology',
        color_discrete_map=tech_colors,
        template='plotly_white',
        title=comp_title,
        labels={'dataset':'Dataset', 'Optimal Capacity':'Installed Capacity (MW)', 'technology':'Technology'}
    )
    fig_comp_absolute.update_layout(barmode='stack', legend_title_text='Technology',
                           margin=dict(l=40, r=20, t=60, b=60))
    fig_comp_absolute.update_xaxes(categoryorder='array', categoryarray=dataset_order)
    fig_comp_absolute.update_yaxes(rangemode='tozero')

    # ---------- 1) 100% stacked composition (SHARE) ----------
    # For each (dataset, technology, __timeslice__), keep the row with the highest Optimal Capacity (by __timeslice__)
    idx = (
        df.groupby(['dataset', 'technology', '__timeslice__'])['Optimal Capacity']
        .idxmax()
        .dropna()
        .astype(int)
    )
    filtered_df = df.loc[idx].reset_index(drop=True)
    cap_df = filtered_df.groupby(['dataset', 'technology'], as_index=False)['Optimal Capacity'].sum()

    # Calculate share per dataset
    cap_df['share'] = cap_df.groupby('dataset')['Optimal Capacity'].transform(lambda x: x / x.sum())

    comp_title = f"Installed Capacity Share by Technology — Year {year}"
    fig_comp = px.bar(
        cap_df,
        x='dataset', y='share', color='technology',
        color_discrete_map=tech_colors,
        template='plotly_white',
        title=comp_title,
        labels={'dataset':'Dataset', 'share':'Installed Capacity Share', 'technology':'Technology'}
    )
    fig_comp.update_layout(barmode='stack', legend_title_text='Technology',
                           margin=dict(l=40, r=20, t=60, b=60))
    fig_comp.update_xaxes(categoryorder='array', categoryarray=dataset_order)
    fig_comp.update_yaxes(rangemode='tozero', tickformat='.0%', title='Installed Capacity Share')

    # ---------- 1b) 100% stacked composition (SHARE) per timeslice ----------
    # desired facet order (4 columns left→right)
    DATASET_ORDER = ['centralized', 'yearly', 'monthly', 'weekly']

    def belongs_to_dataset(ts: str, ds: str) -> bool:
        ts = str(ts).upper()
        if ds == 'centralized':
            return ts == 'C'
        if ds == 'yearly':
            return bool(re.fullmatch(r'Y\d+|Y', ts))
        if ds == 'monthly':
            return bool(re.fullmatch(r'M\d+', ts))
        if ds == 'weekly':
            return bool(re.fullmatch(r'W\d+', ts))
        return True

    def ts_sort_key(ts: str) -> int:
        ts = str(ts).upper()
        if ts == 'C': 
            return 0
        m = re.match(r'[A-Z]+(\d+)$', ts)
        return int(m.group(1)) if m else 10_000

    if '__timeslice__' in df.columns:
        # pick max row per (dataset, tech, country, timeslice)
        idx_ts = (
            df.groupby(['dataset', 'technology', 'country', '__timeslice__'])['Optimal Capacity']
            .idxmax().dropna().astype(int)
        )
        filtered_df_ts = df.loc[idx_ts].reset_index(drop=True)

        # keep only relevant timeslices for each dataset
        filtered_df_ts = filtered_df_ts[
            filtered_df_ts.apply(lambda r: belongs_to_dataset(r['__timeslice__'], r['dataset']), axis=1)
        ]

        # aggregate
        cap_df_ts = filtered_df_ts.groupby(
            ['dataset', '__timeslice__', 'technology'], as_index=False
        )['Optimal Capacity'].sum()

        # drop empty timeslices (per dataset)
        totals = cap_df_ts.groupby(['dataset', '__timeslice__'], as_index=False)['Optimal Capacity'].sum()
        non_empty = totals[totals['Optimal Capacity'] > 0][['dataset', '__timeslice__']]
        cap_df_ts = cap_df_ts.merge(non_empty, on=['dataset', '__timeslice__'], how='inner')

        # compute shares
        cap_df_ts['share'] = cap_df_ts.groupby(['dataset', '__timeslice__'])['Optimal Capacity'] \
                                    .transform(lambda s: s / s.sum())

        # ordering
        cap_df_ts['dataset'] = pd.Categorical(cap_df_ts['dataset'], categories=DATASET_ORDER, ordered=True)
        cap_df_ts['__ts_order__'] = cap_df_ts['__timeslice__'].map(ts_sort_key)
        cap_df_ts = cap_df_ts.sort_values(['dataset', '__ts_order__', 'technology'])

        import plotly.subplots as sp
        # Create 4 subplots, one for each dataset in DATASET_ORDER, with column widths 1:1:12:52
        col_widths = [1, 1, 12, 52]
        fig_comp_ts_subplots = sp.make_subplots(
            rows=1, cols=4,
            shared_yaxes=True,
            subplot_titles=[ds.capitalize() for ds in DATASET_ORDER],
            horizontal_spacing=0.04,
            column_widths=[w / sum(col_widths) for w in col_widths]
        )

        for i, dataset in enumerate(DATASET_ORDER, start=1):
            cap_df_ts_dataset = cap_df_ts[cap_df_ts['dataset'] == dataset]
            if cap_df_ts_dataset.empty:
                continue
            ts_sorted = sorted(cap_df_ts_dataset['__timeslice__'].unique(), key=ts_sort_key)
            for tech in cap_df_ts_dataset['technology'].unique():
                tech_df = cap_df_ts_dataset[cap_df_ts_dataset['technology'] == tech]
                fig_comp_ts_subplots.add_bar(
                    x=tech_df['__timeslice__'],
                    y=tech_df['share'],
                    name=tech,
                    marker_color=tech_colors.get(tech, '#888888'),
                    legendgroup=tech,
                    showlegend=(i == 1),
                    row=1, col=i
                )
                fig_comp_ts_subplots.update_xaxes(
                categoryorder='array',
                categoryarray=ts_sorted,
                row=1, col=i
                )

        fig_comp_ts_subplots.update_layout(
            barmode='stack',
            legend_title_text='Technology',
            margin=dict(l=40, r=20, t=60, b=60),
            title_text=f"Installed Capacity Share by Technology per Timeslice — Year {year}"
        )
        fig_comp_ts_subplots.update_yaxes(rangemode='tozero', tickformat='.0%', title='Installed Capacity Share', row=1, col=1)
    else:
        fig_comp_ts_subplots = None

    # ---------- 1c) 100% stacked composition (ABSOLUTE) per timeslice ----------
    if '__timeslice__' in df.columns:
        import plotly.subplots as sp
        col_widths = [1, 1, 12, 52]
        fig_comp_ts_abs_subplots = sp.make_subplots(
            rows=1, cols=4,
            shared_yaxes=True,
            subplot_titles=[ds.capitalize() for ds in DATASET_ORDER],
            horizontal_spacing=0.04,
            column_widths=[w / sum(col_widths) for w in col_widths]
        )

        for i, dataset in enumerate(DATASET_ORDER, start=1):
            cap_df_ts_dataset = cap_df_ts[cap_df_ts['dataset'] == dataset]
            if cap_df_ts_dataset.empty:
                continue
            ts_sorted = sorted(cap_df_ts_dataset['__timeslice__'].unique(), key=ts_sort_key)
            for tech in cap_df_ts_dataset['technology'].unique():
                tech_df = cap_df_ts_dataset[cap_df_ts_dataset['technology'] == tech]
                fig_comp_ts_abs_subplots.add_bar(
                    x=tech_df['__timeslice__'],
                    y=tech_df['Optimal Capacity'],
                    name=tech,
                    marker_color=tech_colors.get(tech, '#888888'),
                    legendgroup=tech,
                    showlegend=(i == 1),
                    row=1, col=i
                )
                fig_comp_ts_abs_subplots.update_xaxes(
                    categoryorder='array',
                    categoryarray=ts_sorted,
                    row=1, col=i
                )

        fig_comp_ts_abs_subplots.update_layout(
            barmode='stack',
            legend_title_text='Technology',
            margin=dict(l=40, r=20, t=60, b=60),
            title_text=f"Installed Capacity by Technology per Timeslice — Year {year}"
        )
        fig_comp_ts_abs_subplots.update_yaxes(rangemode='tozero', title='Installed Capacity (MW)', row=1, col=1)
    else:
        fig_comp_ts_abs_subplots = None

    # Add to returned figures
    # (add after 'composition_share_per_timeslice')


    # ---------- 1d) 100% stacked composition (CAPEX) per timeslice ----------
    if '__timeslice__' in df.columns:
        import plotly.subplots as sp
        col_widths = [1, 1, 12, 52]
        fig_comp_ts_capex_subplots = sp.make_subplots(
            rows=1, cols=4,
            shared_yaxes=True,
            subplot_titles=[ds.capitalize() for ds in DATASET_ORDER],
            horizontal_spacing=0.04,
            column_widths=[w / sum(col_widths) for w in col_widths]
        )

        for i, dataset in enumerate(DATASET_ORDER, start=1):
            cap_df_ts_dataset = cap_df_ts[cap_df_ts['dataset'] == dataset]
            if cap_df_ts_dataset.empty:
                continue
            ts_sorted = sorted(cap_df_ts_dataset['__timeslice__'].unique(), key=ts_sort_key)
            for tech in cap_df_ts_dataset['technology'].unique():
                # For each (dataset, timeslice, technology), sum Capital Expenditure
                tech_mask = (
                    (df['dataset'] == dataset) &
                    (df['technology'] == tech) &
                    (df['__timeslice__'].isin(ts_sorted))
                )
                tech_df = df[tech_mask].groupby(['__timeslice__'], as_index=False)['Capital Expenditure'].sum()
                fig_comp_ts_capex_subplots.add_bar(
                    x=tech_df['__timeslice__'],
                    y=tech_df['Capital Expenditure'],
                    name=tech,
                    marker_color=tech_colors.get(tech, '#888888'),
                    legendgroup=tech,
                    showlegend=(i == 1),
                    row=1, col=i
                )
                fig_comp_ts_capex_subplots.update_xaxes(
                    categoryorder='array',
                    categoryarray=ts_sorted,
                    row=1, col=i
                )

        fig_comp_ts_capex_subplots.update_layout(
            barmode='stack',
            legend_title_text='Technology',
            margin=dict(l=40, r=20, t=60, b=60),
            title_text=f"Capital Expenditure by Technology per Timeslice — Year {year}"
        )
        fig_comp_ts_capex_subplots.update_yaxes(rangemode='tozero', title='Capital Expenditure (USD)', row=1, col=1)
    else:
        fig_comp_ts_capex_subplots = None

    # ---------- 1e) 100% stacked composition (OPEX) per timeslice ----------

    if '__timeslice__' in df.columns:
        import plotly.subplots as sp
        col_widths = [1, 1, 12, 52]
        fig_comp_ts_opex_subplots = sp.make_subplots(
            rows=1, cols=4,
            shared_yaxes=True,
            subplot_titles=[ds.capitalize() for ds in DATASET_ORDER],
            horizontal_spacing=0.04,
            column_widths=[w / sum(col_widths) for w in col_widths]
        )

        for i, dataset in enumerate(DATASET_ORDER, start=1):
            cap_df_ts_dataset = cap_df_ts[cap_df_ts['dataset'] == dataset]
            if cap_df_ts_dataset.empty:
                continue
            ts_sorted = sorted(cap_df_ts_dataset['__timeslice__'].unique(), key=ts_sort_key)
            for tech in cap_df_ts_dataset['technology'].unique():
                # For each (dataset, timeslice, technology), sum Operational Expenditure
                tech_mask = (
                    (df['dataset'] == dataset) &
                    (df['technology'] == tech) &
                    (df['__timeslice__'].isin(ts_sorted))
                )
                tech_df = df[tech_mask].groupby(['__timeslice__'], as_index=False)['Operational Expenditure'].sum()
                fig_comp_ts_opex_subplots.add_bar(
                    x=tech_df['__timeslice__'],
                    y=tech_df['Operational Expenditure'],
                    name=tech,
                    marker_color=tech_colors.get(tech, '#888888'),
                    legendgroup=tech,
                    showlegend=(i == 1),
                    row=1, col=i
                )
                fig_comp_ts_opex_subplots.update_xaxes(
                    categoryorder='array',
                    categoryarray=ts_sorted,
                    row=1, col=i
                )

        fig_comp_ts_opex_subplots.update_layout(
            barmode='stack',
            legend_title_text='Technology',
            margin=dict(l=40, r=20, t=60, b=60),
            title_text=f"Operational Expenditure by Technology per Timeslice — Year {year}"
        )
        fig_comp_ts_opex_subplots.update_yaxes(rangemode='tozero', title='Operational Expenditure (USD)', row=1, col=1)
    else:
        fig_comp_ts_opex_subplots = None

    


    # ---------- 2) Diverging deltas vs baseline ----------
    # Build wide table and deltas (unchanged)
    # Sum Operational Expenditure per technology, country, and dataset
    idx = (
        df.groupby(['dataset', 'technology', 'country'])['Optimal Capacity']
        .idxmax()
        .dropna()
        .astype(int)
    )
    filtered_df = df.loc[idx].reset_index(drop=True)
    wide = filtered_df.pivot_table(index='technology', columns='dataset', values='Optimal Capacity', aggfunc='sum').fillna(0.0)
    deltas = []
    for d in dataset_order:
        if d == baseline:
            continue
        delta_abs = (wide.get(d, 0.0) - wide.get(baseline, 0.0)).rename('delta')
        sub = delta_abs.reset_index().rename(columns={'index': 'technology'})
        sub['dataset'] = d
        base_vals = wide.get(baseline, 0.0)
        sub['delta_pct'] = sub['delta'] / base_vals.replace(0, np.nan)
        deltas.append(sub)

    delta_df = pd.concat(deltas, ignore_index=True) if deltas else pd.DataFrame(
        columns=['technology', 'dataset', 'delta', 'delta_pct']
    )

    # ---------- FILTER & ORDER ----------
    # Keep only technologies with absolute change > 1 MW  # <<<
    threshold_mw = 1.0
    delta_df['abs_delta'] = delta_df['delta'].abs()      # <<<
    delta_df_filtered = delta_df[delta_df['abs_delta'] > threshold_mw].copy()  # <<<

    # Sort technologies by the largest absolute delta (among datasets)  # <<<
    order_by = (
        delta_df_filtered.groupby('technology')['abs_delta']
        .max()
        .sort_values(ascending=False)
        .index.tolist()
    )

    # ---------- PLOT ----------
    # We plot ABSOLUTE change on a LOG y-axis so it can start at 1 MW.          # <<<
    # The signed delta is still available in the tooltip via customdata/hover.   # <<<
    if delta_df_filtered.empty:
        # Create an empty figure with a helpful title if nothing passes the filter
        delta_title = f"Differences vs {baseline.title()} — Year {year} (no tech > {threshold_mw} MW)"
        fig_delta = px.bar(
            delta_df_filtered,
            x='technology', y='abs_delta', color='dataset',
            template='plotly_white', title=delta_title,
            labels={'abs_delta': '│Δ│ Installed Capacity (MW)', 'technology': 'Technology', 'dataset': 'Scenario'}
        )
    else:
        delta_title = f"Differences vs {baseline.title()} — Year {year} (│Δ│ > {threshold_mw} MW)"
        # We'll show absolute delta on y, and keep signed delta in hover.        # <<<
        fig_delta = px.bar(
            delta_df_filtered,
            x='technology',
            y='abs_delta',               # <<<
            color='dataset',
            template='plotly_white',
            title=delta_title,
            labels={'abs_delta': '│Δ│ Installed Capacity (MW)', 'technology': 'Technology', 'dataset': 'Scenario'},
            hover_data={
                'technology': True,
                'dataset': True,
                'abs_delta': ':.2f',
                'delta': ':.2f',         # signed delta in tooltip               # <<<
                'delta_pct': ':.1%'      # % vs baseline (NaN if baseline=0)
            }
        )

    fig_delta.update_layout(
        barmode='group',
        legend_title_text='Scenario',
        margin=dict(l=40, r=20, t=60, b=80)
    )

    # Make y-axis logarithmic starting at 1 MW                                     # <<<
    if not delta_df_filtered.empty:
        ymax = max(1.0, float(delta_df_filtered['abs_delta'].max()))
        upper_decade = 10 ** math.ceil(math.log10(ymax))
        # On log axes, 'range' is in log10 units; 0 -> 10^0 = 1
        fig_delta.update_yaxes(type='log', range=[0, math.log10(upper_decade)], tickformat=',.2s')
    else:
        fig_delta.update_yaxes(type='log', range=[0, 1])  # safe default

    # Order technologies by |delta| and angle labels                                # <<<
    fig_delta.update_xaxes(categoryorder='array', categoryarray=order_by,)
    fig_delta.update_yaxes(type='log')

    # ---------- 3) Capex and opex ----------
    op_exp_df = (
        df.groupby(['dataset', 'technology', 'country', '__timeslice__'], as_index=False)['Operational Expenditure']
        .max()
    )
    idx = (
        df.groupby(['dataset', 'technology', '__timeslice__'])['Capital Expenditure']
        .idxmax()
        .dropna()
        .astype(int)
    )
    filtered_df = df.loc[idx].reset_index(drop=True)
    cap_exp_df = (
        filtered_df.groupby(['dataset', 'technology', '__timeslice__'], as_index=False)['Capital Expenditure']
        .max()
    )
    exp_df = pd.merge(cap_exp_df, op_exp_df, on=['dataset', 'technology', '__timeslice__'], how='outer').fillna(0.0)

    exp_df = exp_df.groupby(['dataset', 'technology'], as_index=False)[['Capital Expenditure', 'Operational Expenditure']].max()

    # Melt for stacked bar plotting
    exp_long = exp_df.melt(
        id_vars=['dataset', 'technology'],
        value_vars=['Capital Expenditure', 'Operational Expenditure'],
        var_name='Expenditure Type',
        value_name='Expenditure (USD)'
    )

    # Consistent tech color
    exp_colors = custom_palette

    fig_exp = px.bar(
        exp_long,
        x='dataset',
        y='Expenditure (USD)',
        color='technology',
        facet_col='Expenditure Type',
        color_discrete_map=exp_colors,
        barmode='stack',
        template='plotly_white',
        title=f"Capital and Operational Expenditure by Dataset — Year {year}",
        labels={'dataset': 'Dataset', 'Expenditure (USD)': 'Expenditure (USD)', 'technology': 'Technology'}
    )
    fig_exp.update_layout(margin=dict(l=40, r=20, t=60, b=60))
    fig_exp.update_xaxes(categoryorder='array', categoryarray=dataset_order)

    return {
        'composition_share_absolute': fig_comp_absolute,
        'composition_share': fig_comp,
        'composition_share_per_timeslice': fig_comp_ts_subplots,
        'composition_absolute_per_timeslice': fig_comp_ts_abs_subplots,
        'composition_capex_per_timeslice': fig_comp_ts_capex_subplots,
        'composition_opex_per_timeslice': fig_comp_ts_opex_subplots,
        'delta_vs_baseline': fig_delta,
        'expenditure': op_exp_df,
        'expenditure_stacked': fig_exp,
    }

In [ ]:
def plot_last_iteration_timeslices(
    dataframes_yearly,
    dataframes_monthly,
    dataframes_weekly,
    year=None,
    width=1400,
    height_per_country=320,
    max_iteration=99,
    countries=None
):
    # ---------- helpers
    def try_int(x):
        try:
            return int(x)
        except Exception:
            return x

    def pick_year(dy, dm, dw, year):
        all_years = set()
        for d in (dy, dm, dw):
            if isinstance(d, dict):
                all_years.update([y for y in d.keys() if isinstance(y, (int, str))])
        if not all_years:
            raise ValueError("No years found in the provided dataframes.")
        all_years_int = [try_int(y) for y in all_years]
        if year is None:
            # default: max available year
            chosen = max(all_years_int)
        else:
            chosen = try_int(year)
            if chosen not in all_years_int:
                raise ValueError(f"Requested year {year} not found. Available: {sorted(all_years_int)}")
        return int(chosen)

    def last_iteration_block(droot, year, timescale_key):
        """
        For 'Y': return the dict for the last iteration under droot[year]['Y'] (should be a single 'Y' key).
        For 'M' or 'W': return a dict mapping timeslice (e.g., 'M1', 'M2', ...) to the last iteration's block for each timeslice.
        """
        if not droot or year not in droot:
            return None
        year_block = droot[year]
        if timescale_key == 'Y':
            if 'Y' not in year_block:
                return None
            iterations_block = year_block['Y']
            if not iterations_block:
                return None
            last_iter_key = sorted(iterations_block.keys(), key=try_int)[-1]
            return iterations_block[last_iter_key]
        elif timescale_key in ('M', 'W'):
            # Collect all timeslices for this timescale
            ts_prefix = timescale_key
            ts_blocks = {}
            for ts_key, ts_val in year_block.items():
                if isinstance(ts_key, str) and ts_key.startswith(ts_prefix):
                    # Each ts_val is a dict of iterations
                    if not ts_val:
                        ts_blocks[ts_key] = None
                        continue
                    last_iter_key = sorted(ts_val.keys(), key=try_int)[-1]
                    ts_blocks[ts_key] = ts_val[last_iter_key]
            return ts_blocks
        else:
            return None

    def collect_countries_from_df(df):
        if df is None or not isinstance(df, pd.DataFrame) or df.empty:
            return []
        return list(df.index)

    def compute_imp_exp_for_country(tdf, country):
        """Imports: sum of exchange > 0 where end_country == country.
           Exports: sum of exchange > 0 where start_country == country.
        """
        if tdf is None or not isinstance(tdf, pd.DataFrame) or tdf.empty:
            return 0.0, 0.0
        exp = tdf.loc[(tdf['start_country'] == country) & (tdf['exchange'] > 0), 'exchange'].sum()
        imp = tdf.loc[(tdf['end_country'] == country) & (tdf['exchange'] > 0), 'exchange'].sum()
        imp = 0.0 if pd.isna(imp) else float(imp)
        exp = 0.0 if pd.isna(exp) else float(exp)
        return imp, exp

    def demand0(df, country):
        if df is None or not isinstance(df, pd.DataFrame) or df.empty:
            return 0.0
        if 'demand_0' in df.columns:
            val = df.at[country, 'demand_0'] if country in df.index else 0.0
            return 0.0 if pd.isna(val) else float(val)
        # fallback if needed
        for c in ('demand', 'demand0'):
            if c in df.columns:
                val = df.at[country, c] if country in df.index else 0.0
                return 0.0 if pd.isna(val) else float(val)
        return 0.0

    # ---------- collect last-iteration data blocks for the chosen year
    Y = pick_year(dataframes_yearly, dataframes_monthly, dataframes_weekly, year)

    y_last = last_iteration_block(dataframes_yearly, Y, 'Y')
    m_last = last_iteration_block(dataframes_monthly, Y, 'M')
    w_last = last_iteration_block(dataframes_weekly, Y, 'W')

    # Yearly: expect a single df + tdf
    y_df   = y_last.get('df') if isinstance(y_last, dict) else None
    y_tdf  = y_last.get('transmission_df') if isinstance(y_last, dict) else None
    y_k = y_last.get('k') if isinstance(y_last, dict) else None
    # Monthly/Weekly can be stored per timeslice inside the iteration block. We’ll try to normalize:
    # We accept either:
    #   m_last = {'M1': {'df':..., 'transmission_df':...}, 'M2': {...}, ...}
    # or       = list/iterable of dicts with a 'timeslice' key
    def normalize_timeslice_map(block, prefix):
        ts_map = {}
        if block is None:
            return ts_map
        # dict keyed by timeslice
        if isinstance(block, dict) and any(k.startswith(prefix) for k in block.keys() if isinstance(k, str)):
            for ts, payload in block.items():
                if isinstance(payload, dict) and 'df' in payload:
                    ts_map[ts] = {
                        'df': payload.get('df'),
                        'tdf': payload.get('transmission_df'),
                        'k': payload.get('k')
                    }
        else:
            # maybe a flat dict with many entries having 'timeslice' key
            if isinstance(block, dict):
                candidates = block.values()
            elif isinstance(block, (list, tuple)):
                candidates = block
            else:
                candidates = []

            for item in candidates:
                if isinstance(item, dict):
                    ts = item.get('timeslice')
                    if isinstance(ts, str) and ts.startswith(prefix):
                        ts_map[ts] = {
                            'df': item.get('df'),
                            'tdf': item.get('transmission_df'),
                            'k': item.get('k')
                        }
        return ts_map

    m_ts_map = normalize_timeslice_map(m_last, 'M')
    w_ts_map = normalize_timeslice_map(w_last, 'W')

    # Ensure complete sets of timeslices with None if missing
    months = [f"M{i}" for i in range(1, 13)]
    weeks  = [f"W{i}" for i in range(1, 53)]
    for m in months:
        m_ts_map.setdefault(m, {'df': None, 'tdf': None, 'k': None})
    for w in weeks:
        w_ts_map.setdefault(w, {'df': None, 'tdf': None, 'k': None})

    # ---------- use the provided countries list for row order and titles
    if not countries:
        raise ValueError("No countries found across yearly/monthly/weekly last iterations.")
        # ---------- prep the subplot grid with custom column widths (1:12:52)        #
        rows = len(countries)

    rows = len(countries)
    cols = 3  # Y | M | W
    total = 1 + 12 + 52
    col_widths = [1/total, 12/total, 52/total]

    # Titles: one per subplot, in row-major order (Y, M, W for each country)
    subplot_titles = []
    for i, c in enumerate(countries):
        if i == 0:
            subplot_titles.extend([
                f"<span style='font-size:14px'>Yearly</span>",
                f"<span style='font-size:14px'>Monthly</span>",
                f"<span style='font-size:14px'>Weekly</span>"
            ])
        else:
            subplot_titles.extend(["", "", ""])

    fig = make_subplots(
        rows=rows,
        cols=cols,
        subplot_titles=subplot_titles,
        horizontal_spacing=0.03,
        vertical_spacing=0.02,
        column_widths=col_widths
    )

        # Add country names as y-axis titles for the first (yearly) subplot in each row
    yaxis_titles = {i + 1: c for i, c in enumerate(countries)}
    for row, country in yaxis_titles.items():
        fig.update_yaxes(title_text=country, row=row, col=1, title_standoff=10, automargin=True)

    # colors for the three series
    demand_color = "#1f77b4"  # default Plotly blue
    import_color = "#2ca02c"  # green
    export_color = "#ff7f0e"  # orange
    # ---------- build per-country panels (one row per country, columns: yearly | monthly | weekly)
    for i, country in enumerate(countries):
        row = i + 1

        # ---- YEARLY PANEL (single timeslice 'Y')
        y_x = ['Y']
        y_dem = [demand0(y_df, country)]
        y_imp, y_exp = compute_imp_exp_for_country(y_tdf, country)
        y_imports = [y_imp]
        y_exports = [y_exp]

        yearly_subset = pd.DataFrame({
            'Iteration': y_x,
            'Demand': y_dem,
            'Import': y_imports,
            'Export': y_exports
        })

        fig.add_trace(go.Bar(
            x=yearly_subset['Iteration'], y=yearly_subset['Demand'],
            name='Production',
            marker_color='blue', showlegend=(row == 1),
            offsetgroup='demand', legendgroup='Demand'
        ), row=row, col=1, secondary_y=False)

        fig.add_trace(go.Bar(
            x=yearly_subset['Iteration'], y=yearly_subset['Export'],
            name='Export',
            marker=dict(color='blue', pattern=dict(shape='\\')),
            offsetgroup='demand', base=0,
            showlegend=(row == 1), legendgroup='Export'
        ), row=row, col=1, secondary_y=False)

        fig.add_trace(go.Bar(
            x=yearly_subset['Iteration'], y=yearly_subset['Import'],
            name='Import',
            marker_color='green', opacity=0.8,
            offsetgroup='import', base=yearly_subset['Demand'],
            showlegend=(row == 1), legendgroup='Import'
        ), row=row, col=1, secondary_y=False)

        if y_k >= max_iteration:
            fig.add_vrect(
                x0=-0.5, x1=0.5,
                fillcolor="gray", opacity=0.5, line_width=0,
                row=row, col=1
            )

        # ---- MONTHLY PANEL (M1..M12)
        m_x = months
        m_dem, m_imp, m_exp = [], [], []
        for m in months:
            df_m = m_ts_map[m]['df']
            tdf_m = m_ts_map[m]['tdf']
            m_dem.append(demand0(df_m, country))
            mi, me = compute_imp_exp_for_country(tdf_m, country)
            m_imp.append(mi)
            m_exp.append(me)

        monthly_subset = pd.DataFrame({
            'Iteration': m_x,
            'Demand': m_dem,
            'Import': m_imp,
            'Export': m_exp
        })

        fig.add_trace(go.Bar(
            x=monthly_subset['Iteration'], y=monthly_subset['Demand'],
            name='Production',
            marker_color='blue', showlegend=False,
            offsetgroup='demand', legendgroup='Demand'
        ), row=row, col=2, secondary_y=False)

        fig.add_trace(go.Bar(
            x=monthly_subset['Iteration'], y=monthly_subset['Export'],
            name='Export',
            marker=dict(color='blue', pattern=dict(shape='\\')),
            offsetgroup='demand', base=0,
            showlegend=False, legendgroup='Export'
        ), row=row, col=2, secondary_y=False)

        fig.add_trace(go.Bar(
            x=monthly_subset['Iteration'], y=monthly_subset['Import'],
            name='Import',
            marker_color='green', opacity=0.8,
            offsetgroup='import', base=monthly_subset['Demand'],
            showlegend=False, legendgroup='Import'
        ), row=row, col=2, secondary_y=False)

        # Add gray rectangle if monthly iteration exceeds max_iteration
        for idx, m in enumerate(months):
            m_k = m_ts_map[m]['k']
            if m_k is not None and m_k >= max_iteration:
                fig.add_vrect(
                    x0=idx - 0.5, x1=idx + 0.5,
                    fillcolor="gray", opacity=0.5, line_width=0,
                    row=row, col=2
                )

        # ---- WEEKLY PANEL (W1..W52)
        w_x = weeks
        w_dem, w_imp, w_exp = [], [], []
        for w in weeks:
            df_w = w_ts_map[w]['df']
            tdf_w = w_ts_map[w]['tdf']
            w_dem.append(demand0(df_w, country))
            wi, we = compute_imp_exp_for_country(tdf_w, country)
            w_imp.append(wi)
            w_exp.append(we)

        weekly_subset = pd.DataFrame({
            'Iteration': w_x,
            'Demand': w_dem,
            'Import': w_imp,
            'Export': w_exp
        })

        fig.add_trace(go.Bar(
            x=weekly_subset['Iteration'], y=weekly_subset['Demand'],
            name='Production',
            marker_color='blue', showlegend=False,
            offsetgroup='demand', legendgroup='Demand'
        ), row=row, col=3, secondary_y=False)

        fig.add_trace(go.Bar(
            x=weekly_subset['Iteration'], y=weekly_subset['Export'],
            name='Export',
            marker=dict(color='blue', pattern=dict(shape='\\')),
            offsetgroup='demand', base=0,
            showlegend=False, legendgroup='Export'
        ), row=row, col=3, secondary_y=False)

        fig.add_trace(go.Bar(
            x=weekly_subset['Iteration'], y=weekly_subset['Import'],
            name='Import',
            marker_color='green', opacity=0.8,
            offsetgroup='import', base=weekly_subset['Demand'],
            showlegend=False, legendgroup='Import'
        ), row=row, col=3, secondary_y=False)

        # Add gray rectangle if weekly iteration exceeds max_iteration
        for idx, w in enumerate(weeks):
            w_k = w_ts_map[w]['k']
            if w_k is not None and w_k >= max_iteration:
                fig.add_vrect(
                    x0=idx - 0.5, x1=idx + 0.5,
                    y0=0, y1=1.31,
                    fillcolor="gray", opacity=0.5, line_width=0,
                    row=row, col=3
                )

    # Update layout for less space between columns and shared x-axes
    fig.update_layout(
        template="plotly_white",
        barmode="stack",
        width=width,
        height=max(500, rows * height_per_country),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.8),
        margin=dict(t=90, b=60, l=40, r=20)
    )

    # Reduce horizontal spacing between columns
    #fig.update_layout(horizontal_spacing=0.01)

    # Shared x-axes for each column
    for col in range(1, 4):
        fig.update_xaxes(matches=f'x{col}', row=1, col=col)

    # Only show x-axis labels at the last row
    for row in range(1, rows + 1):
        showticklabels = (row == rows)
        for col in range(1, 4):
            fig.update_xaxes(showticklabels=showticklabels, row=row, col=col)
            if showticklabels:
                if col == 1:
                    fig.update_xaxes(row=row, col=col, tickangle=35)
                elif col == 2:
                    fig.update_xaxes(row=row, col=col, tickangle=45)
                elif col == 3:
                    fig.update_xaxes(row=row, col=col, tickangle=45)
            else:
                fig.update_xaxes(title_text=None, row=row, col=col)
                fig.update_xaxes(title_text=None, row=row, col=col)

    # Make weekly axis labels a bit lighter to avoid clutter
    fig.update_xaxes(tickangle=0, tickfont=dict(size=9), row=1, col=3)

    scaled_font=30
    fig.update_layout(
        title=dict( font=dict(size=scaled_font)),
        font=dict(size=scaled_font),  # affects legend & tick labels
        showlegend=False,
    )



    # --- Subplot titles ---
    for ann in fig['layout']['annotations']:
        ann['font'] = dict(size=scaled_font)
    fig.show()

    # Use to_image for interactive environments, or write_image for file output.
    # If write_image is too slow or times out, try reducing figure size or complexity.
    fig.show()
    

In [17]:
def plot_demand_transmission(data, width=1200, height=900, timeslice=None, year=None):

    # Sort iterations numerically if possible
    def try_int(x):
        try:
            return int(x)
        except Exception:
            return x

    iterations = sorted(data.keys(), key=try_int)
    records = []

    for k in iterations:
        df = data[k]['df']
        tdf = data[k].get('transmission_df', pd.DataFrame())

        for country in df.index:
            demand = df.at[country, f'demand_0'] if f'demand_0' in df.columns else 0.0
            imports = -tdf.loc[(tdf['start_country'] == country) & (tdf['exchange'] < 0), 'exchange'].sum()
            exports = tdf.loc[(tdf['start_country'] == country) & (tdf['exchange'] > 0), 'exchange'].sum()
            total_demand = demand
            imports = imports if not pd.isna(imports) else 0.0
            exports = exports if not pd.isna(exports) else 0.0
            cost = df.at[country, '0'] if '0' in df.columns else None
            cost_per_unit = cost / total_demand if total_demand and cost is not None else None
            cost_per_unit_with_exchange = df.at[country, 'total_cost_after_exchange'] / (total_demand + imports) if total_demand and 'total_cost_after_exchange' in df.columns else None
            records.append({
                'Iteration': try_int(k),
                'Country': country,
                'Demand': demand,
                'Import': imports,
                'Export': exports,
                'CostPerUnit': cost_per_unit,
                'CostPerUnitWithExchange': cost_per_unit_with_exchange,
            })

    df_tx = pd.DataFrame(records)
    df_tx = df_tx.sort_values('Iteration')
    countries = df_tx['Country'].unique()
    cols = 4
    rows = (len(countries) + cols - 1) // cols
    # Use the same y-axis range for all subplots    #
    fig = make_subplots(
        rows=rows, cols=cols,
        subplot_titles=countries,
        shared_xaxes=False, shared_yaxes=False,
        vertical_spacing=0.12, horizontal_spacing=0.05,
        specs=[[{"secondary_y": True} for _ in range(cols)] for _ in range(rows)]
    )

    for i, country in enumerate(countries):
        row = i // cols + 1
        col = i % cols + 1
        subset = df_tx[df_tx['Country'] == country].sort_values('Iteration')

        fig.add_trace(go.Bar(
            x=subset['Iteration'], y=subset['Demand'],
            name='Production',
            marker_color='blue', showlegend=(i == 0),
            offsetgroup='demand', legendgroup='Demand'
        ), row=row, col=col, secondary_y=False)

        fig.add_trace(go.Bar(
            x=subset['Iteration'], y=subset['Export'],
            name='Export',
            marker=dict(color='blue', pattern=dict(shape='\\')),
            offsetgroup='demand', base=0,
            showlegend=(i == 0), legendgroup='Export'
        ), row=row, col=col, secondary_y=False)

        fig.add_trace(go.Bar(
            x=subset['Iteration'], y=subset['Import'],
            name='Import',
            marker_color='green', opacity=0.8,
            offsetgroup='import', base=subset['Demand'],
            showlegend=(i == 0), legendgroup='Import'
        ), row=row, col=col, secondary_y=False)

        fig.add_trace(go.Scatter(
            x=subset['Iteration'], y=subset['CostPerUnitWithExchange'],
            mode='lines+markers', name='Cost per Unit (with Exchange)',
            line=dict(color='red', dash='dash', width=6),
            showlegend=(i == 0), legendgroup='CostPerUnitWithExchange'
        ), row=row, col=col, secondary_y=True)

    fig.update_layout(
        height=height, width=width,
        barmode='overlay',
        legend=dict(orientation="h", yanchor="bottom", y=-0.1, xanchor="center", x=0.5)
    )
    #fig.update_yaxes(title_text="Energy", secondary_y=False)
    #fig.update_yaxes(title_text="Cost per Unit", secondary_y=True)

    fig.update_yaxes(title_text="MWh", row=2, col=1)
    fig.update_yaxes(title_text="$/MWh", secondary_y=True, row=2, col=4)
    fig.update_layout(template="plotly_white")

    # os.makedirs(f"{SAVING_PATH}/convergence", exist_ok=True)
    # fig.write_html(
    #     f"{SAVING_PATH}/convergence/exchange_over_iterations_{timeslice}_{year}.html",
    #     include_plotlyjs='cdn'
    # )
    scaled_font=30
    fig.update_layout(
        title=dict( font=dict(size=scaled_font)),
        font=dict(size=scaled_font),  # affects legend & tick labels
    )

    # --- Subplot titles ---
    for ann in fig['layout']['annotations']:
        ann['font'] = dict(size=scaled_font)
    fig.show()

In [11]:
# plot_demand_transmission(
#                 dataframes_monthly[2030]['M6'],
#                 width=2070*3,
#                 height=700*3,
#             )

In [19]:
plot_last_iteration_timeslices(
    dataframes_yearly,
    dataframes_monthly,
    dataframes_weekly,
    year=2030,                
    width=2000*3,
    height_per_country=100*3,
    max_iteration=99,
    countries=countries,
)

ValueError: 
    Invalid value of type 'builtins.str' received for the 'showlegend' property of layout
        Received value: 'false'

    The 'showlegend' property must be specified as a bool
    (either True, or False)

In [18]:
figs = build_better_comparison_plots(
    dataframes_yearly=dataframes_yearly,
    dataframes_monthly=dataframes_monthly,
    dataframes_weekly=dataframes_weekly,
    dataframes_centralized=dataframes_centralized,
    year=2030,             # your target year
    top_n_for_slope=8,     # show top-8 most different technologies
    baseline='centralized', # deltas are vs this dataset
    initial_capacity_df=initial_capacity_df, # for filtering out technologies never built
)

figs['composition_share_absolute'].update_layout(width=700, height=600)
figs['composition_share_absolute'].show()  # absolute stacked to compare scale
figs['composition_share'].update_layout(width=700, height=600)
figs['composition_share'].show()   # 100% stacked to compare mix
#figs['composition_share'].write_image(f"{SAVING_PATH}/composition_share_2030.png", scale=3)
if figs['composition_share_per_timeslice']:
    figs['composition_share_per_timeslice'].update_layout(width=1400, height=300)
    figs['composition_share_per_timeslice'].show()   # 100% stacked per timeslice to compare mix
    figs['composition_absolute_per_timeslice'].update_layout(width=1400, height=300)
    figs['composition_absolute_per_timeslice'].show()   # absolute stacked per timeslice to compare scale
    figs['composition_capex_per_timeslice'].update_layout(width=1400, height=300)
    figs['composition_capex_per_timeslice'].show()   # capex stacked per timeslice to compare scale
    figs['composition_opex_per_timeslice'].update_layout(width=1400, height=300)
    figs['composition_opex_per_timeslice'].show()
figs['expenditure_stacked'].update_layout(width=900, height=600)
figs['expenditure_stacked'].update_layout(showlegend=False)
#figs['expenditure_stacked'].write_image(f"{SAVING_PATH}/expenditure_stacked_2030.png", scale=3)
figs['expenditure_stacked'].show()  # stacked capex/opex per dataset





In [14]:
initial_capacity_df


Biomass                    79.83700
Coal                    49373.73159
Combined-Cycle Gas       6593.15773
Nuclear                  1940.00000
Oil                       532.43902
Onshore Wind             3416.46669
Run of River             6075.69869
Solar                    6607.03807
Pumped Hydro Storage     2892.00000
Reservoir & Dam         13995.92357
Name: Installed Capacity, dtype: float64

In [15]:
ghj

NameError: name 'ghj' is not defined

In [ ]:
df = figs.copy()

In [ ]:
df[(df['dataset'] == 'monthly') & (df['technology'] == 'Solar')].sum()

technology                 SolarSolarSolarSolarSolarSolarSolarSolarSolarS...
country                                             AOBWCDLSMWMZNASZTZZAZMZW
Optimal Capacity                                                193688.54085
Installed Capacity                                                6605.36257
Supply                                                       296559667.49072
Capital Expenditure                                        9850216350.452749
Operational Expenditure                                        5785958.11303
Capacity Factor                                                     1.980206
Curtailment                                                    5698198.05353
__timeslice__                                     M4M9M11M8M12M9M5M8M4M4M7M9
dataset                    monthlymonthlymonthlymonthlymonthlymonthlymont...
dtype: object

In [ ]:
df['technology'].unique()

array(['Combined-Cycle Gas', 'Onshore Wind', 'Run of River', 'Solar',
       'H2 electrolysis', 'H2 fuel cell', 'battery charger',
       'battery discharger', 'Reservoir & Dam', 'Battery Storage',
       'Hydrogen Storage', 'Coal', 'Oil', 'Biomass', 'Nuclear',
       'Pumped Hydro Storage'], dtype=object)

In [ ]:
op_exp_df = (
        df.groupby(['technology', 'country', 'dataset'], as_index=False)['Operational Expenditure']
        .sum()
    )
op_exp_df.sort_values(['Operational Expenditure'], ascending=False).head(20)

,technology,country,dataset,Operational Expenditure
80,Combined-Cycle Gas,ZA,weekly,1.402696e+10
64,Coal,ZW,weekly,1.272606e+10
73,Combined-Cycle Gas,MZ,weekly,1.214361e+10
57,Coal,ZA,monthly,1.160884e+10
77,Combined-Cycle Gas,TZ,weekly,1.046198e+10
56,Coal,SAPP,centralized,9.914645e+09
58,Coal,ZA,weekly,9.762311e+09
59,Coal,ZA,yearly,9.288899e+09
79,Combined-Cycle Gas,ZA,monthly,6.652600e+09
198,Nuclear,ZA,weekly,4.587459e+09


In [ ]:
idx = (
        df.groupby(['dataset', 'technology', 'country'])['Optimal Capacity']
        .idxmax()
        .dropna()
        .astype(int)
    )
filtered_df = df.loc[idx].reset_index(drop=True)
capex_sum = filtered_df.groupby(['dataset', 'technology'], as_index=False)['Capital Expenditure'].sum()
capex_sum.sort_values(['Capital Expenditure'], ascending=False).head(20)




,dataset,technology,Capital Expenditure
41,weekly,Onshore Wind,8.510516e+10
25,monthly,Onshore Wind,2.285975e+10
45,weekly,Solar,2.187680e+10
38,weekly,Hydrogen Storage,2.029936e+10
29,monthly,Solar,9.497144e+09
34,weekly,Coal,6.138547e+09
2,centralized,Coal,6.048868e+09
50,yearly,Coal,6.048868e+09
18,monthly,Coal,6.048868e+09
36,weekly,H2 electrolysis,3.848475e+09


In [ ]:
filtered_df

,technology,country,Optimal Capacity,Installed Capacity,Supply,Capital Expenditure,Operational Expenditure,Capacity Factor,Curtailment,__timeslice__,dataset
0,Battery Storage,SAPP,54272.62144,0.00000,1.810528e+07,8.618114e+08,0.000000e+00,0.475350,0.000000e+00,C,centralized
1,Biomass,SAPP,79.83700,79.83700,6.552824e+05,2.219591e+07,9.807751e+06,0.936958,4.408969e+04,C,centralized
2,Coal,SAPP,49373.73159,49373.73159,4.111633e+08,6.048868e+09,9.914645e+09,0.950636,2.135062e+07,C,centralized
3,Combined-Cycle Gas,SAPP,6593.15773,6593.15773,7.328955e+06,5.569182e+08,3.459959e+08,0.126895,5.042711e+07,C,centralized
4,H2 electrolysis,SAPP,3687.20764,0.00000,5.498014e+06,1.799151e+08,7.010871e+04,0.212772,0.000000e+00,C,centralized
...,...,...,...,...,...,...,...,...,...,...,...
413,battery discharger,SZ,90.81616,0.00000,1.406611e+05,0.000000e+00,1.426931e+03,0.196455,0.000000e+00,Y,yearly
414,battery discharger,TZ,512.74405,0.00000,8.411706e+05,0.000000e+00,8.494502e+03,0.208083,0.000000e+00,Y,yearly
415,battery discharger,ZA,6393.23486,0.00000,8.783888e+06,0.000000e+00,9.964805e+04,0.174269,0.000000e+00,Y,yearly
416,battery discharger,ZM,74.43810,0.00000,1.271520e+05,0.000000e+00,1.275634e+03,0.216661,0.000000e+00,Y,yearly


In [ ]:
df[(df['dataset'] == 'monthly') & (df['technology'] == 'H2 electrolysis') & (df['__timeslice__'] == 'M8')]

,technology,country,Optimal Capacity,Installed Capacity,Supply,Capital Expenditure,Operational Expenditure,Capacity Factor,Curtailment,__timeslice__,dataset
1469,H2 electrolysis,AO,3290.87020,0.0,6.419300e+06,1.605761e+08,79508.40115,0.278345,0.0,M8,monthly
1481,H2 electrolysis,BW,425.75833,0.0,9.424579e+05,2.077463e+07,11130.26512,0.315867,0.0,M8,monthly
1491,H2 electrolysis,CD,1587.87494,0.0,3.303864e+06,7.747942e+07,42195.17061,0.296901,0.0,M8,monthly
1499,H2 electrolysis,LS,138.12945,0.0,2.762867e+05,6.739945e+06,3425.98161,0.285417,0.0,M8,monthly
1506,H2 electrolysis,MW,34.30554,0.0,6.579516e+04,1.673919e+06,779.23804,0.273675,0.0,M8,monthly
1517,H2 electrolysis,MZ,0.00000,0.0,6.000000e-04,1.161000e-02,0.00001,0.000000,0.0,M8,monthly
1527,H2 electrolysis,NA,228.69129,0.0,4.685890e+05,1.115886e+07,5560.39294,0.292380,0.0,M8,monthly
1538,H2 electrolysis,SZ,471.26752,0.0,8.133489e+05,2.299522e+07,9802.57609,0.246272,0.0,M8,monthly
1550,H2 electrolysis,TZ,872.63626,0.0,1.719311e+06,4.257977e+07,20701.00349,0.281143,0.0,M8,monthly
1564,H2 electrolysis,ZA,14686.75739,0.0,3.414136e+07,7.166317e+08,434034.34909,0.331712,0.0,M8,monthly


In [ ]:
filtered_df[filtered_df['technology']=='Onshore Wind'].sort_values(['dataset', 'country'])

,technology,country,Optimal Capacity,Installed Capacity,Supply,Capital Expenditure,Operational Expenditure,Capacity Factor,Curtailment,__timeslice__,dataset
9,Onshore Wind,SAPP,7755.11216,3416.46669,1.370963e+07,8.475994e+08,3.442533e+05,0.201806,9.542914e+05,C,centralized
83,Onshore Wind,AO,1221.53678,0.00000,1.279477e+06,1.335086e+08,3.201311e+04,0.119570,3.608537e+05,M7,monthly
84,Onshore Wind,BW,2946.48815,0.00000,5.326414e+06,3.220381e+08,1.321896e+05,0.206360,2.697720e+06,M9,monthly
85,Onshore Wind,CD,0.00000,0.00000,0.000000e+00,1.550000e-03,0.000000e+00,0.000000,0.000000e+00,M1,monthly
86,Onshore Wind,LS,0.00000,0.00000,0.000000e+00,1.000000e-05,0.000000e+00,0.000000,0.000000e+00,M10,monthly
87,Onshore Wind,MW,1399.11861,0.00000,2.203399e+06,1.529175e+08,5.490887e+04,0.179777,5.459722e+05,M9,monthly
88,Onshore Wind,MZ,16095.70840,0.00000,2.740624e+07,1.759190e+09,6.817247e+05,0.194373,1.026132e+07,M9,monthly
89,Onshore Wind,NA,587.15111,1.70576,1.055589e+06,6.417302e+07,2.626416e+04,0.205230,1.691607e+05,M9,monthly
90,Onshore Wind,SZ,193.35526,0.00000,2.140267e+05,2.113287e+07,5.174668e+03,0.126359,6.553567e+04,M8,monthly
91,Onshore Wind,TZ,5432.36570,1.14479,9.970059e+06,5.937335e+08,2.431124e+05,0.209510,7.768907e+05,M7,monthly


In [ ]:
cap_exp_df = (
        df.groupby(['technology', 'country', 'dataset'], as_index=False)['Capital Expenditure']
        .max()
    )
cap_exp_df

,technology,country,dataset,Capital Expenditure
0,Battery Storage,AO,monthly,1.138046e+02
1,Battery Storage,AO,weekly,2.700000e-03
2,Battery Storage,AO,yearly,1.751693e+08
3,Battery Storage,BW,monthly,1.836051e+02
4,Battery Storage,BW,weekly,1.110824e+02
...,...,...,...,...
413,battery discharger,ZM,weekly,0.000000e+00
414,battery discharger,ZM,yearly,0.000000e+00
415,battery discharger,ZW,monthly,0.000000e+00
416,battery discharger,ZW,weekly,0.000000e+00


In [ ]:
hydrogen_storage_sum = df[df['technology'] == 'Hydrogen Storage'] \
    .groupby('dataset')['Optimal Capacity'] \
    .sum() \
    .reset_index() \
    .sort_values('Optimal Capacity', ascending=False)
hydrogen_storage_sum

,dataset,Optimal Capacity
2,weekly,2.549035e+07
1,monthly,3.809430e+06
0,centralized,1.880307e+05
3,yearly,1.832887e+05


In [ ]:
df

,technology,country,Optimal Capacity,Installed Capacity,Supply,Capital Expenditure,Operational Expenditure,Capacity Factor,Curtailment,__timeslice__,dataset
0,Battery Storage,SAPP,54272.62144,0.00000,1.810528e+07,8.618114e+08,0.000000e+00,0.475350,0.000000e+00,C,centralized
1,Biomass,SAPP,79.83700,79.83700,6.552824e+05,2.219591e+07,9.807751e+06,0.936958,4.408969e+04,C,centralized
2,Coal,SAPP,49373.73159,49373.73159,4.111633e+08,6.048868e+09,9.914645e+09,0.950636,2.135062e+07,C,centralized
3,Combined-Cycle Gas,SAPP,6593.15773,6593.15773,7.328955e+06,5.569182e+08,3.459959e+08,0.126895,5.042711e+07,C,centralized
4,H2 electrolysis,SAPP,3687.20764,0.00000,5.498014e+06,1.799151e+08,7.010871e+04,0.212772,0.000000e+00,C,centralized
...,...,...,...,...,...,...,...,...,...,...,...
413,battery discharger,SZ,90.81616,0.00000,1.406611e+05,0.000000e+00,1.426931e+03,0.196455,0.000000e+00,Y,yearly
414,battery discharger,TZ,512.74405,0.00000,8.411706e+05,0.000000e+00,8.494502e+03,0.208083,0.000000e+00,Y,yearly
415,battery discharger,ZA,6393.23486,0.00000,8.783888e+06,0.000000e+00,9.964805e+04,0.174269,0.000000e+00,Y,yearly
416,battery discharger,ZM,74.43810,0.00000,1.271520e+05,0.000000e+00,1.275634e+03,0.216661,0.000000e+00,Y,yearly


In [ ]:
df[df['dataset'] == 'monthly']['technology'].unique()

array(['Battery Storage', 'Biomass', 'Coal', 'Combined-Cycle Gas',
       'H2 electrolysis', 'H2 fuel cell', 'Hydrogen Storage', 'Nuclear',
       'Oil', 'Onshore Wind', 'Pumped Hydro Storage', 'Reservoir & Dam',
       'Run of River', 'Solar', 'battery charger', 'battery discharger'],
      dtype=object)

In [ ]:
df[(df['dataset'] == 'monthly') & (df['technology'] == 'Nuclear')]

,technology,country,Optimal Capacity,Installed Capacity,Supply,Capital Expenditure,Operational Expenditure,Capacity Factor,Curtailment,__timeslice__,dataset
79,Nuclear,ZA,1940.0,1940.0,9.193364e+06,8.555350e+08,1.554707e+08,0.540964,7.801036e+06,M1,monthly


In [ ]:
df['dataset'].unique()

array(['yearly', 'monthly', 'weekly', 'centralized'], dtype=object)

In [ ]:
df[df['technology'] == 'Hydrogen Storage'] \
    .groupby(['__timeslice__'])['Optimal Capacity'] \
    .sum() \
    .reset_index() \
    .sort_values('Optimal Capacity', ascending=False)

,__timeslice__,Optimal Capacity
18,W14,1.094670e+07
31,W26,8.105615e+06
27,W22,7.310358e+06
36,W30,6.813545e+06
14,W10,6.371102e+06
...,...,...
1,M1,1.030016e+06
40,W34,1.023886e+06
39,W33,9.847603e+05
0,C,1.880307e+05


In [ ]:
df[df['dataset'] == 'monthly']

,technology,country,Optimal Capacity,Installed Capacity,Supply,Capital Expenditure,Operational Expenditure,Capacity Factor,Curtailment,__timeslice__,dataset
135,Combined-Cycle Gas,AO,960.68000,960.68000,1.870826e+04,8.114780e+07,883200.18169,0.002223,8.396849e+06,M1,monthly
136,Onshore Wind,AO,0.00000,0.00000,0.000000e+00,1.660000e-03,0.00000,0.000000,0.000000e+00,M1,monthly
137,Run of River,AO,5352.03873,5352.03873,4.402036e+06,1.450085e+09,47306.95636,0.093892,2.459161e+06,M1,monthly
138,Solar,AO,12446.31030,309.17131,1.268233e+07,6.329690e+08,249940.93979,0.116320,3.590873e+06,M1,monthly
139,H2 electrolysis,AO,3320.73911,0.00000,6.823953e+06,1.620335e+08,84869.32234,0.293229,0.000000e+00,M1,monthly
...,...,...,...,...,...,...,...,...,...,...,...
1726,battery charger,ZW,0.00242,0.00000,5.389750e+00,9.321883e+01,0.05856,0.280992,0.000000e+00,M9,monthly
1727,battery discharger,ZW,0.00269,0.00000,4.888980e+00,0.000000e+00,0.05297,0.230483,0.000000e+00,M9,monthly
1728,Reservoir & Dam,ZW,740.44586,740.44586,5.477347e+05,0.000000e+00,5433.57070,0.084445,5.938571e+06,M9,monthly
1729,Battery Storage,ZW,0.00558,0.00000,1.374700e-01,8.854892e+01,0.00000,0.494624,0.000000e+00,M9,monthly
